## Примеры работы с pylatex


In [ ]:
%pip install pylatex

In [ ]:
from pylatex import Document, Section, Subsection, Command
from pylatex.utils import italic, NoEscape


def fill_document(doc):
    """Add a section, a subsection and some text to the document.

    :param doc: the document
    :type doc: :class:`pylatex.document.Document` instance
    """
    with doc.create(Section("A section")):
        doc.append("Some regular text and some ")
        doc.append(italic("italic text. "))

        with doc.create(Subsection("A subsection")):
            doc.append("Also some crazy characters: $&#{}")


### Basic document


In [ ]:
# Basic document
doc = Document("basic")
fill_document(doc)

doc.generate_pdf(clean_tex=False)
doc.generate_tex()

# Document with `\maketitle` command activated
doc = Document()

doc.preamble.append(Command("title", "Awesome Title"))
doc.preamble.append(Command("author", "Anonymous author"))
doc.preamble.append(Command("date", NoEscape(r"\today")))
doc.append(NoEscape(r"\maketitle"))

fill_document(doc)

doc.generate_pdf("basic_maketitle", clean_tex=False)

# Add stuff to the document
with doc.create(Section("A second section")):
    doc.append("Some text.")

doc.generate_pdf("basic_maketitle2", clean_tex=False)
tex = doc.dumps()  # The document as string in LaTeX syntax


### Multirow example


In [ ]:
from pylatex import Document, Section, Subsection, Tabular, MultiColumn, MultiRow

doc = Document("multirow")
section = Section("Multirow Test")

test1 = Subsection("MultiColumn")
test2 = Subsection("MultiRow")
test3 = Subsection("MultiColumn and MultiRow")
test4 = Subsection("Vext01")

table1 = Tabular("|c|c|c|c|")
table1.add_hline()
table1.add_row((MultiColumn(4, align="|c|", data="Multicolumn"),))
table1.add_hline()
table1.add_row((1, 2, 3, 4))
table1.add_hline()
table1.add_row((5, 6, 7, 8))
table1.add_hline()
row_cells = ("9", MultiColumn(3, align="|c|", data="Multicolumn not on left"))
table1.add_row(row_cells)
table1.add_hline()

table2 = Tabular("|c|c|c|")
table2.add_hline()
table2.add_row((MultiRow(3, data="Multirow"), 1, 2))
table2.add_hline(2, 3)
table2.add_row(("", 3, 4))
table2.add_hline(2, 3)
table2.add_row(("", 5, 6))
table2.add_hline()
table2.add_row((MultiRow(3, data="Multirow2"), "", ""))
table2.add_empty_row()
table2.add_empty_row()
table2.add_hline()

table3 = Tabular("|c|c|c|")
table3.add_hline()
table3.add_row(
    (MultiColumn(2, align="|c|", data=MultiRow(2, data="multi-col-row")), "X")
)
table3.add_row((MultiColumn(2, align="|c|", data=""), "X"))
table3.add_hline()
table3.add_row(("X", "X", "X"))
table3.add_hline()

table4 = Tabular("|c|c|c|")
table4.add_hline()
col1_cell = MultiRow(4, data="span-4")
col2_cell = MultiRow(2, data="span-2")
table4.add_row((col1_cell, col2_cell, "3a"))
table4.add_hline(start=3)
table4.add_row(("", "", "3b"))
table4.add_hline(start=2)
table4.add_row(("", col2_cell, "3c"))
table4.add_hline(start=3)
table4.add_row(("", "", "3d"))
table4.add_hline()

test1.append(table1)
test2.append(table2)
test3.append(table3)
test4.append(table4)

section.append(test1)
section.append(test2)
section.append(test3)
section.append(test4)

doc.append(section)
doc.generate_pdf(clean_tex=False)


## Генерация таблицы по актам


In [ ]:
%pip install jira
%pip install dataclasses-json

In [ ]:
from datetime import datetime
from jira_dtos import UserWorklogItemDto


t = UserWorklogItemDto(
    task_code="2,",
    task_name="kekw",
    start_period_date=datetime.now(),
    end_period_date=datetime.now(),
    time_spent_seconds=234234,
)

print(t)


In [ ]:
from jira import JIRA
import json

from jira_dtos import JiraInfo, DateRange
from jira_helpers import map_user_worklog, map_issues


In [ ]:
# NOTE: a sensitive data
user_login = "JIRA LOGIN"
user_jira_password = "PASSWORD"
jira_info = JiraInfo(
    jira_server_address="https://jira.com/",
    basic_auth=(user_login, user_jira_password),
    user_login=user_login,
)


In [ ]:
auth_jira = JIRA(
    basic_auth=jira_info.basic_auth, options={"server": jira_info.jira_server_address}
)


In [ ]:
startWorklogDate = "2022-08-14"
endWorklogDate = "2022-09-14"
jql = f'worklogAuthor in ("{jira_info.user_login}") and worklogDate >= {startWorklogDate} and worklogDate <= {endWorklogDate}'
found_issues = auth_jira.search_issues(jql)
print(found_issues)


In [ ]:
from time import strptime
import dateutil.parser

start_date = dateutil.parser.isoparse(startWorklogDate)
end_date = dateutil.parser.isoparse(endWorklogDate)
worklogs_data_range = DateRange(start_date=start_date, end_date=end_date)
map_issue = lambda issue: map_issues(issue, jira_info.user_login, worklogs_data_range)
user_worklogs = map(map_issue, found_issues)
user_worklog_items = list(map(map_user_worklog, user_worklogs))


In [ ]:
t1 = UserWorklogItemDto.schema().dumps(user_worklog_items, many=True)
print(t1)
t2 = UserWorklogItemDto.schema().loads(t1, many=True)
t3 = list(filter(None, t2))
print()
print(t3[0].task_code)


In [ ]:
in_hours = filter(lambda x: x != None, user_worklog_items)
in_hours = map(lambda x: x.time_spent_seconds, in_hours)  # type: ignore
s = sum(in_hours)
print(s / 3600)


In [ ]:
from dataclasses import dataclass
from dataclasses_json import dataclass_json


@dataclass_json
@dataclass
class Person:
    name: str


people_json = [Person("lidatong")]
t = Person.schema().dumps(people_json, many=True)  # '[{"name": "lidatong"}]'
t2 = Person.schema().loads('[{"name2": "lidatong"}]', many=True)

print(type(t2[0]))


In [ ]:
# generate_act_doc(user_worklog_items)


## Пример работы с питоновскими датами -> str и обратно


In [ ]:
%pip install python-dateutil

In [ ]:
import dateutil.parser

raw_date = "2022-08-14T15:59:00.000+0300"
date = dateutil.parser.isoparse(raw_date)
print(date)
print(type(date))

str_date = date.isoformat()
print(str_date)

str2_date = "2022-08-14"
date2 = dateutil.parser.isoparse(str2_date)
print(date2)

is_intersected_yep = date2.timestamp() <= date.timestamp()
print(is_intersected_yep)


## Пример генерации PDF актов через pylatex


In [ ]:
%pip install pylatex

In [ ]:
from dataclasses import dataclass
from typing import final
from pylatex import (
    Document,
    Section,
    Subsection,
    Tabular,
    MultiColumn,
    MultiRow,
    Command,
)
from jira_helpers import MappedUserWorklog
from jira_dtos import UserWorklogItemDto
from pylatex.utils import bold


@final
@dataclass
class ActRecord:
    column_number: int
    service_name: str
    time_spent_in_hours: float


MappedUserWorklogs = list[MappedUserWorklog]
ActRecords = list[ActRecord]


def map_act_record(worklog: UserWorklogItemDto, index: int) -> ActRecord:
    service_name = f"{worklog.task_code}. {worklog.task_name}"
    in_hours = worklog.time_spent_seconds / 3600
    record = ActRecord(
        column_number=index, service_name=service_name, time_spent_in_hours=in_hours
    )

    return record


def format_act_records(worklogs: MappedUserWorklogs) -> ActRecords:
    filtered = filter(None, worklogs)
    enumerated = enumerate(filtered)
    act_records = map(lambda x: map_act_record(x[1], x[0]), enumerated)
    act_records = list(act_records)

    return act_records


In [ ]:
from dataclasses import dataclass
from typing import final
from pylatex import (
    Document,
    Section,
    Subsection,
    Tabular,
    MultiColumn,
    MultiRow,
    Command,
)
from jira_helpers import MappedUserWorklog
from jira_dtos import UserWorklogItemDto
from pylatex.utils import bold


@final
@dataclass
class TaskRecord:
    column_number: int
    service_name: str
    validity: str


TaskRecords = list[TaskRecord]


def map_task_record(worklog: UserWorklogItemDto, index: int) -> TaskRecord:
    service_name = f"{worklog.task_code}. {worklog.task_name}"
    start_period = worklog.start_period_date.strftime("%d.%m.%Y")
    end_period = worklog.end_period_date.strftime("%d.%m.%Y")
    validity = f"{start_period} - {end_period}"
    record = TaskRecord(
        column_number=index, service_name=service_name, validity=validity
    )

    return record


def format_task_records(worklogs: MappedUserWorklogs) -> TaskRecords:
    filtered = filter(None, worklogs)
    enumerated = enumerate(filtered)
    task_records = map(lambda x: map_task_record(x[1], x[0]), enumerated)
    task_records = list(task_records)

    return task_records


In [ ]:
from datetime import datetime

time = datetime.now().strftime("%d.%m.%Y")
print("time:", time)


In [ ]:
def generate_act_table(worklogs: MappedUserWorklogs) -> Tabular:
    table = Tabular("| p | p | p |", row_height=1.5)
    table.add_hline()
    header = [
        "№",
        MultiColumn(size=1, align="|r|", data="Наименование услуг (объем, перечень)"),
        "Затраченное время (часы)",
    ]
    table.add_row(header, mapper=[bold])
    table.add_hline()

    formatted_records = format_act_records(worklogs)
    for record in formatted_records:
        cells = [record.column_number, record.service_name, record.time_spent_in_hours]
        table.add_row(cells)
        table.add_hline()

    row_cells = [
        MultiColumn(size=2, align="|r|", data="Стоимость, руб.: 331 703, 40 руб. "),
        173.7,
    ]
    table.add_row(row_cells, mapper=[bold])
    table.add_hline()

    return table


def generate_act_doc(worklogs) -> None:
    doc = Document("акты", page_numbers=True, font_size="13pt")
    doc.preamble.append(Command("usepackage", "babel", "russian"))
    doc.preamble.append(Command("usepackage", "graphicx"))
    doc.preamble.append(Command("usepackage", "array"))
    doc.preamble.append(Command("usepackage", "lipsum"))
    doc.preamble.append(Command("usepackage", "geometry"))
    doc.preamble.append(
        Command("geometry", "{a4paper, left=10mm, right=10mm, top=20mm")
    )

    section = Section("АКТ СДАЧИ ПРИЕМКИ")

    test1 = Subsection("Пример таблицы")
    table1 = generate_act_table(worklogs)
    test1.append(table1)
    section.append(test1)

    doc.append(section)
    doc.generate_pdf(clean_tex=False, compiler="lualatex")


In [ ]:
def generate_task_table(worklogs: MappedUserWorklogs) -> Tabular:
    table = Tabular("| p | p | p |", row_height=1.5)
    table.add_hline()
    header = [
        "№",
        MultiColumn(size=1, align="|с|", data="Наименование услуг"),
        "Период выполнения работ",
    ]
    table.add_hline()

    formatted_records = format_task_records(worklogs)
    for record in formatted_records:
        cells = [record.column_number, record.service_name, record.validity]
        table.add_row(cells)
        table.add_hline()

    row_cells = [
        MultiColumn(size=2, align="|r|", data="Стоимость, руб.: 331 703, 40 руб. "),
        173.7,
    ]
    table.add_row(row_cells, mapper=[bold])
    table.add_hline()

    return table


def generate_task_doc(worklogs) -> None:
    doc = Document("задание", page_numbers=True, font_size="13pt")
    doc.preamble.append(Command("usepackage", "babel", "russian"))
    doc.preamble.append(Command("usepackage", "graphicx"))
    doc.preamble.append(Command("usepackage", "array"))
    doc.preamble.append(Command("usepackage", "lipsum"))
    doc.preamble.append(Command("usepackage", "geometry"))
    doc.preamble.append(
        Command("geometry", "{a4paper, left=10mm, right=10mm, top=20mm")
    )

    section = Section("Задание на выполенине работ")

    test1 = Subsection("Пример таблицы")
    table1 = generate_task_table(worklogs)
    test1.append(table1)
    section.append(test1)

    doc.append(section)
    doc.generate_pdf(clean_tex=False, compiler="lualatex")


In [ ]:
worklogs_str = '[{"task_name":"Что есть зло? Чтобы это не было, оно возникает из слабости","task_code":"D2C-1","time_spent_seconds":52200,"start_period_date":1662987567.499,"end_period_date":1663078476.16},{"task_name":"Требуется много таланта и умения, чтобы скрыть свой талант и умение","task_code":"D2C-2","time_spent_seconds":126000,"start_period_date":1662473799.184,"end_period_date":1663047038.527},{"task_name":"Человек - это животное, которое умеет совершать сделки. Ни одна собака не будет меняться костью с другой собакой","task_code":"D2C-3","time_spent_seconds":225000,"start_period_date":1661522651.363,"end_period_date":1662722506.429},{"task_name":"Мы не должны расстраиваться, когда другие скрывают правду от нас, так как мы скрываем правду и от себя тоже","task_code":"D2C-4","time_spent_seconds":115200,"start_period_date":1660831746.787,"end_period_date":1661497110.483},{"task_name":"У каждого человека внутри сидит предатель","task_code":"D2C-10","time_spent_seconds":99000,"start_period_date":1660573388.942,"end_period_date":1660813309.804}]'

In [ ]:
from jira_dtos import UserWorklogItemDto


worklogs = UserWorklogItemDto.schema().loads(worklogs_str, many=True)

generate_act_doc(worklogs)


In [ ]:
from jira_dtos import UserWorklogItemDto


worklogs = UserWorklogItemDto.schema().loads(worklogs_str, many=True)

generate_task_doc(worklogs)


## Пример генерации pdf Акта через Lualatex


In [ ]:
import subprocess

file_path = "./test_acts.tex"
job_name = "--job-name=kekw"
output_directory = "--output-directory=generated_pdfs"
subprocess.run(["lualatex", job_name, output_directory, file_path])


## Формирование отчета на основе жсон данных и шаблона в Jinja


### Формирование данных по Заданию


In [ ]:
import subprocess
from pathlib import Path

FilePath = str


def generate_pdf_acts(tex_file_path: Path) -> Path:
    tex_file_name = tex_file_path.stem
    job_name = f"--job-name={tex_file_name}"
    generated_pdf_folder = "generated_pdfs"
    output_directory = f"--output-directory={generated_pdf_folder}"
    tex_file_pdf_folder = Path(generated_pdf_folder, tex_file_name)
    generated_pdf_file_path = get_file_path_in_cur_dir(tex_file_pdf_folder)
    text_file_filesystem_path = str(tex_file_path)
    t = [job_name, output_directory, text_file_filesystem_path]
    print(t)
    subprocess.run(["lualatex", job_name, output_directory, text_file_filesystem_path])

    return generated_pdf_file_path
